In [ ]:
import numpy as np
from scipy.sparse import csr_matrix as csr
from scipy.sparse import eye
import random

In [ ]:
from prettytable import PrettyTable

def print_matrix(A):
    k, n = A.shape
    p = PrettyTable()
    for i in range(k):
        for j in range(k):
            A[i][j] = round(A[i][j], 3)
        p.add_row(A[i])
    print(p.get_string(header=False, border=False))
    print()

In [ ]:
pip install PrettyTable

# LU Decomposition

In [ ]:
def LU_decompse(A): # LU разложение Вход 2х мерный массив, Выход 2х мерные массивы
    n_rows = len(A)
    n_cols = len(A[0])
    L = np.eye(n_rows, n_cols)
    U = np.zeros((n_rows, n_cols))
    for i in range(n_rows):
        for j in range(n_cols):
            if i <= j:
                _s = 0
                for k in range(i):
                    _s += L[i][k] * U[k][j]
                U[i][j] = A[i][j] - _s
            else:
                _s = 0
                for k in range(j):
                    _s += L[i][k] * U[k][j]
                L[i][j] = (A[i][j] - _s) / U[j][j]
    return L, U

In [ ]:
def LU_decompse_csr(A): # LU разложение Вход csr матрица Выход csr матрица (не оптимально)
    n_rows, n_cols = A.shape
    L = eye(n_rows, n_cols, format="csr")
    U = csr((n_rows, n_cols), dtype=float)
    for i in range(n_rows):
        for j in range(n_cols):
            if i <= j:
                _s = 0
                for k in range(i):
                    _s += L[i, k] * U[k, j]
                U[i, j] = A[i, j] - _s
            else:
                _s = 0
                for k in range(j):
                    _s += L[i, k] * U[k, j]
                L[i, j] = (A[i, j] - _s) / U[j, j]
    return L, U

In [ ]:
def LU_decompse_csr_full(A):
    n_rows, n_cols = A.shape
    l_data = []
    l_indices = []
    l_indptr = []
    u_data = []
    u_indices = []
    u_indptr = []
    for i in range(n_rows):
        l_indptr.append(len(l_data))
        u_indptr.append(len(u_data))
        for j in range(n_cols):
            if i <= j:
                _s = 0
                if i == j:
                    l_data.append(1)
                    l_indices.append(j)
                for x, y in zip(l_data[l_indptr[i]:], list(map(lambda x: x[0], filter(lambda x: x[1] == j, zip(u_data, u_indices))))):
                    _s += x * y
                u_data.append(A[i, j] - _s)
                u_indices.append(j)
            else:
                _s = 0
                for x, y in zip(l_data[l_indptr[i]:], list(map(lambda x: x[0], filter(lambda x: x[1] == j, zip(u_data, u_indices))))):
                    _s += x * y
                l_data.append((A[i, j] - _s) / u_data[u_indptr[j]])
                l_indices.append(j)
    l_indptr.append(len(l_data))
    u_indptr.append(len(u_data))
    return csr((l_data, l_indices, l_indptr)), csr((u_data, u_indices, u_indptr))

## Example

In [ ]:
A = np.array([
    [6, -3, 5, 0, 2, 0, 0],
    [-4, 0, 7, -3, 0, 2, 0],
    [0, 9, -3, -6, 0, 7, 1],
    [5, -2, 0, 0, 1, 7, -3],
    [-1, 0, 0, 5, 0, 2, 0],
    [9, -8, 7, 0, 2, 3, 0],
    [3, 0, -4, 1, 9, 0, 5]
])
L, U = LU_decompse(A)
print("Matrix A")
print_matrix(A)
print("Matrix L")
print_matrix(L)
print("Matrix U")
print_matrix(U)
B = np.dot(L, U)
print("Matrix L * U")
print_matrix(B)

In [ ]:
A = np.array([[10, -7.6666, 0], 
              [-3, 6, 2], 
              [5, -1, 5]])

print_matrix(A)
# sA = csr(A)
# print("sA")
# print(sA)

# print(LU_decompse(A))
# l, u =  LU_decompse_csr(sA)
# print("L: ", l)
# print("U: ", u)

# l, u = LU_decompse_csr_full(sA)
# print("L: ", l.toarray())
# print("U: ", u.toarray())

# Usage of sparse matrices

In [ ]:
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
mtx = csr((data, (row, col)), shape=(3, 3))
mtx     

In [ ]:
mtx.todense()

In [ ]:
mtx.data

In [ ]:
mtx.indices

In [ ]:
mtx.indptr

# Solving a system of linear equations

## Additional functions

In [ ]:
# get i-th row of sparse matrix A

def get_row(A, i):
    _, n = A.shape
    row = np.zeros(n)
    non_zero_elements_in_row = A.data[A.indptr[i]:A.indptr[i + 1]]
    indices_of_non_zero_elements_in_row = A.indices[A.indptr[i]:A.indptr[i + 1]]
    k = 0
    for i in range(n):
        if k < len(non_zero_elements_in_row) and i == indices_of_non_zero_elements_in_row[k]:
            row[i] = non_zero_elements_in_row[k]
            k += 1
    return row

## Solve function

In [ ]:
# A: sparse matrix
# b: array of values

def solve(A, b):
    L, U = LU_decompse_csr_full(A)
    _, n = A.shape
    
    # solving Ly = b
    y = np.zeros(n)
    for i in range(n):
        row = get_row(L, i)
        y[i] = b[i] - np.dot(y[:i], row[:i])
    
    # solving Ux = y
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        row = get_row(U, i)
        x[i] = (y[i] - np.dot(x[i + 1:], row[i + 1:])) / row[i]
        
    return x

## Example

In [ ]:
A = np.array([[10, -7, 2, -3, 0, 0], 
              [-3, 4, 6, 0, 5, 4], 
              [5, -1, 5, -9, 2, 8],
              [0, 1, -6, 3, 2, 7],
              [0, 0, 0, 5, -9, 3],
              [6, 0, 4, 7, -2, 0]
])
b = [-15, 23, -3, -34, 27, 8]
x = solve(csr(A), b)
print("Matrix A")
print_matrix(A)
print("Vector b\n", b, "\n")
print("Vector x\n ", x, "\n")
print("Vector A * x")
print(np.dot(A, x))

# Finding inverse matrix

In [ ]:
def inverse(A):
    _, n = A.shape
    I = np.eye(n)
    A_inverse = np.zeros(A.shape)
    for i in range(n):
        x = solve(A, I[:, i])
        A_inverse[:, i] = x
    return csr(A_inverse)

## Example

In [ ]:
A = np.array([
    [3, -5, 0, 2, 0, 8],
    [4, 1, 2, 0, 4, 3],
    [1, -1, 3, 0, 2, 0],
    [0, 1, 9, -2, 5, 0],
    [7, -4, -5, 0, 6, -3],
    [0, -4, 2, 8, 0, 7]
])

np.set_printoptions(precision=3)

print("Matrix A")
print_matrix(A)
print("Inverse Matrix A")
A_inverse = inverse(csr(A)).todense()
print(A_inverse, "\n")
print("Matrix E")
E = np.dot(A, A_inverse)
print(E)

# Diagonal Matrices

In [128]:
random.seed()
def generate_diagonal_matrix(k):
    values = [0, -1, -2, -3, -4, -5, -6]
    noise = 10**(-k)
    matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            matrix[i][j] = random.choice(values)
    for i in range(k):
        matrix[i][i] = -(sum(matrix[i]) - matrix[i][i]) + noise
    return matrix

def solve_d_system(k):
    A = generate_diagonal_matrix(k)
#     while abs(np.linalg.det(A)) < 10**(-5):
#         A = generate_diagonal_matrix(k)
    x = [x for x in range(1, k + 1)]
    F = np.dot(A, x)
    x_new = solve(csr(A), F)
    error = np.linalg.norm(x - x_new)
    return x_new, error

def solve_d_systems(k):
#     """
    error_array = []
    x_new_array = []
    for i in range(1, k + 1):
        x_new, error = solve_d_system(i)
        x_new_array.append(x_new)
        error_array.append(error)
        print("k =", i, "\terror =", round(error, 5), "\tx' =", x_new)
#     """

        
solve_d_systems(30)
# generate_diagonal_matrix(3)

k = 1 	error = 0.0 	x' = [1.]
k = 2 	error = 0.0 	x' = [1. 2.]
k = 3 	error = 0.0 	x' = [1. 2. 3.]
k = 4 	error = 0.0 	x' = [1. 2. 3. 4.]
k = 5 	error = 0.0 	x' = [1. 2. 3. 4. 5.]
k = 6 	error = 0.0 	x' = [1. 2. 3. 4. 5. 6.]
k = 7 	error = 0.0 	x' = [1. 2. 3. 4. 5. 6. 7.]
k = 8 	error = 0.0 	x' = [1. 2. 3. 4. 5. 6. 7. 8.]
k = 9 	error = 1e-05 	x' = [1. 2. 3. 4. 5. 6. 7. 8. 9.]
k = 10 	error = 0.00012 	x' = [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
k = 11 	error = 0.00113 	x' = [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
k = 12 	error = 0.00626 	x' = [ 1.002  2.002  3.002  4.002  5.002  6.002  7.002  8.002  9.002 10.002
 11.002 12.002]
k = 13 	error = 0.67356 	x' = [ 1.187  2.187  3.187  4.187  5.187  6.187  7.187  8.187  9.187 10.187
 11.187 12.187 13.187]
k = 14 	error = 7.48331 	x' = [ 3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
k = 15 	error = 58.09475 	x' = [-14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1.
   0.]
k = 16 	error = nan 	x' = [nan nan

<ipython-input-59-bf432800b33b>:18: RuntimeWarning: invalid value encountered in double_scalars
  x[i] = (y[i] - np.dot(x[i + 1:], row[i + 1:])) / row[i]
<ipython-input-59-bf432800b33b>:18: RuntimeWarning: divide by zero encountered in double_scalars
  x[i] = (y[i] - np.dot(x[i + 1:], row[i + 1:])) / row[i]


 23 	error = 59.14859 	x' = [-11.333 -10.333  -9.333  -8.333  -7.333  -6.333  -5.333  -4.333  -3.333
  -2.333  -1.333  -0.333   0.667   1.667   2.667   3.667   4.667   5.667
   6.667   7.667   8.667   9.667  10.667]
k = 24 	error = nan 	x' = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
k = 25 	error = 85.0 	x' = [-1.600e+01 -1.500e+01 -1.400e+01 -1.300e+01 -1.200e+01 -1.100e+01
 -1.000e+01 -9.000e+00 -8.000e+00 -7.000e+00 -6.000e+00 -5.000e+00
 -4.000e+00 -3.000e+00 -2.000e+00 -1.000e+00  1.717e-15  1.000e+00
  2.000e+00  3.000e+00  4.000e+00  5.000e+00  6.000e+00  7.000e+00
  8.000e+00]
k = 26 	error = nan 	x' = [nan inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf]
k = 27 	error = nan 	x' = [ nan  nan -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf]
k = 28 	error = 63.49803 	x' = [-11. -10.  -9.  -8.  -

# Gilbert Matrices

In [ ]:
def generate_gilbert_matrix(k):
    matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            matrix[i][j] = 1 / (i + j + 1)
    return matrix


def solve_system(k):
    A = generate_gilbert_matrix(k)
    x = np.zeros((k))
    for i in range(k):
        x[i] = i + 1
    F = np.dot(A, x)
    x_new = solve(csr(A), F)
    error = np.linalg.norm(x - x_new)
    return x_new, error
    

def solve_systems(k):
    error_array = []
    x_new_array = []
    for i in range(1, k + 1):
        x_new, error = solve_system(i)
        x_new_array.append(x_new)
        error_array.append(error)
        print("k =", i, "\terror =", round(error, 5), "\tx' =", x_new, "\n")
    return error_array, x_new_array
    

solve_systems(15)